In [1]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar"        
    }
}

In [13]:
UNION_DF = spark.sql('''

                        SELECT * FROM precogdb.df5_red_deviation_above_gs
                        where date > '2021-10-31'
                        --WHERE CAST(loadeddate AS DATE) = CAST(current_timestamp() AS DATE)
                        
                        UNION
                        SELECT * FROM precogdb.df5_amber_deviation_above_gs
                        where date > '2021-10-31'
                        --WHERE CAST(loadeddate AS DATE) = CAST(current_timestamp() AS DATE)
                        
                        UNION
                        SELECT * FROM precogdb.df5_green_deviation_above_gs
                        where flightdate > '2021-10-31'
                        --WHERE CAST(loadeddate AS DATE) = CAST(current_timestamp() AS DATE)
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
UNION_DF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------+-------+--------+------+--------------------+-----+------+---------+-------+--------+----------+-------------------+-------------------+------+--------------+------------+-----------+---------+------------+-------------+-------+-------------+---------+------+-----+-------------------+-----------------+---------------------+-------------------+------------+------------+------------+-----------------+--------------------------+--------------------+-----------+--------------+--------------------+------------+------------+--------+----------------------+----------------------------+--------------------------+--------------------+--------------------+----+---------+
|exceedanceid|         exceedance|phaseid|   phase|   ecn|        employeename| rank|tailno|departure|arrival|flightno|      date|          chocksoff|           chockson|domint|manufacturerid|manufacturer|dataframeid|dataframe|aircrafttype|aircraftmodel|payload|blocktime_hrs|occurance|typeid| fl

In [5]:
UNION_DF.createOrReplaceTempView('union_df_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
UNION_DF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17892

In [6]:
PilotInfoDF = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
PilotInfoDF.createOrReplaceTempView('PilotInfoDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
final_df_df5 = spark.sql('''
                            SELECT 
                            CAST(ud.exceedanceid AS INT),
                            ud.exceedance,
                            CAST(ud.phaseid AS INT),
                            ud.phase,
                            ud.ecn,
                            ud.employeename,
                            ud.rank,
                            ud.tailno,
                            ud.departure,
                            ud.arrival,
                            ud.flightno AS FlightNumber,
                            ud.date as FLightDate ,
                            ud.chocksoff,
                            ud.chockson,
                            ud.domint,
                            CAST(ud.manufacturerid AS INT),
                            ud.manufacturer,
                            CAST(ud.dataframeid AS INT),
                            ud.dataframe,
                            ud.aircrafttype,
                            ud.aircraftmodel,
                            ud.payload,
                            ud.blocktime_hrs as `Block time`,
                            occurance as occurence,
                            CAST(typeid AS INT),
                            flag as type,
                            CAST(trigger_start_hours AS DECIMAL(8,2)) AS `Trigger Start Hours`,
                            CAST(trigger_end_hours AS DECIMAL(8,2)) AS `Trigger End Hours`,
                            CAST(occurence_start_hours AS DECIMAL(8,2)) AS `Occurence Start Hours`,
                            CAST(occurence_end_hours AS DECIMAL(8,2)) AS `Occurence End Hours`,
                            CAST(duration_min AS INT) AS `Duration(Sec)`,
                            CAST(`BARO_COR_ALT` AS INT) AS `BARO COR ALT`,
                            CAST(`RADIO_HEIGHT` AS INT) AS `RADIO HEIGHT`,
                            CAST(`COMPUTED_AIRSPEED` AS FLOAT) AS `COMPUTED AIRSPEED`,
                            CAPT_DISPLAY_PITCH_ATTIUDE AS `CAPT. DISPLAY PITCH ATTIUDE`,
                            CAST(`CAPT_DISPLAY_HEADING` AS FLOAT) AS `CAPT. DISPLAY HEADING`,
                            CAST(`GROUNDSPEED` AS FLOAT) AS `GROUND SPEED`,
                            CAST(`VERTICAL_SPEED` AS INT) AS `VERTICAL SPEED`,
                            CAST(`FLAP_HANDLE_POSITION` AS INT) AS `FLAP HANDLE POSITION`,
                            CAST(`TORQUE_ENG_L` AS FLOAT) AS `LEFT ENGINE N1 \ TORQUE ENG L`,
                            CAST(`TORQUE_ENG_R` AS FLOAT) AS `RIGHT ENGINE N1 \ TORQUE ENG R`,
                            CAST('' AS INT) AS `VREF SPEED`,
                            AFCS_Glideslope_Active AS `G/S ENGAGE`,
                            GLIDE_PATH_DEV_PILOT_ABOVE AS `GLIDESLOPE DEVIATION - L`,
                            GLIDE_PATH_DEV_COPILOT_ABOVE AS `GLIDESLOPE DEVIATION - R`,
                            MAIN_WOW AS `LEFT MAIN GEAR AIR/GND`,
                            '' AS `RIGHT MAIN GEAR AIR/GND`,
                            loadeddate AS LoadDate,
                            filename
                        FROM union_df_1 ud
                    
                    
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
final_df_df5.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

17894

In [21]:
final_df_df5 \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog") \
  .option("dbtable", "DeviationAboveGSSTG") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
UNION_DF_1 = spark.sql('''

                        SELECT * FROM precogdb.df6_red_deviation_above_gs
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df6_amber_deviation_above_gs
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df6_green_deviation_above_gs
                        where flightdate > '2021-10-31'
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
UNION_DF_1.createOrReplaceTempView('union_df_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
final_df_df6 = spark.sql('''
                            SELECT 
                            CAST(ud.exceedanceid AS INT),
                            ud.exceedance,
                            CAST(ud.phaseid AS INT),
                            ud.phase,
                            ud.ecn,
                            ud.employeename,
                            ud.rank,
                            ud.tailno,
                            ud.departure,
                            ud.arrival,
                            ud.flightno AS FlightNumber,
                            ud.date as FLightDate ,
                            ud.chocksoff,
                            ud.chockson,
                            ud.domint,
                            CAST(ud.manufacturerid AS INT),
                            ud.manufacturer,
                            CAST(ud.dataframeid AS INT),
                            ud.dataframe,
                            ud.aircrafttype,
                            ud.aircraftmodel,
                            ud.payload,
                            ud.blocktime_hrs as `Block time`,
                            occurance as occurence,
                            CAST(typeid AS INT),
                            flag as type,
                            CAST(trigger_start_hours AS DECIMAL(8,2)) AS `Trigger Start Hours`,
                            CAST(trigger_end_hours AS DECIMAL(8,2)) AS `Trigger End Hours`,
                            CAST(occurence_start_hours AS DECIMAL(8,2)) AS `Occurence Start Hours`,
                            CAST(occurence_end_hours AS DECIMAL(8,2)) AS `Occurence End Hours`,
                            CAST(duration_min AS INT) AS `Duration(Sec)`,
                            CAST(`BARO_COR_ALT` AS INT) AS `BARO COR ALT`,
                            CAST(`RADIO_HEIGHT` AS INT) AS `RADIO HEIGHT`,
                            CAST(`COMPUTED_AIRSPEED` AS FLOAT) AS `COMPUTED AIRSPEED`,
                            CAPT_DISPLAY_PITCH_ATTIUDE AS `CAPT. DISPLAY PITCH ATTIUDE`,
                            CAST(`CAPT_DISPLAY_HEADING` AS FLOAT) AS `CAPT. DISPLAY HEADING`,
                            CAST(`GROUNDSPEED` AS FLOAT) AS `GROUND SPEED`,
                            CAST(`VERTICAL_SPEED` AS INT) AS `VERTICAL SPEED`,
                            CAST(`FLAP_HANDLE_POSITION` AS INT) AS `FLAP HANDLE POSITION`,
                            CAST(`TORQUE_ENG_L` AS FLOAT) AS `LEFT ENGINE N1 \ TORQUE ENG L`,
                            CAST(`TORQUE_ENG_R` AS FLOAT) AS `RIGHT ENGINE N1 \ TORQUE ENG R`,
                            CAST('' AS INT) AS `VREF SPEED`,
                            AFCS_GS_MODE AS `G/S ENGAGE`,
                            ILS_GS_DEV_1 AS `GLIDESLOPE DEVIATION - L`,
                            ILS_GS_DEV_2 AS `GLIDESLOPE DEVIATION - R`,
                            WOW_MLG AS `LEFT MAIN GEAR AIR/GND`,
                            '' AS `RIGHT MAIN GEAR AIR/GND`,
                            loadeddate AS LoadDate,
                            filename
                    FROM union_df_1 ud
                    
                    
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
final_df_df6 \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog") \
  .option("dbtable", "DeviationAboveGSSTG") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…